# Rendering
In this tutorial, we'll explain the ins and outs of rendering: the process of creating an image from 3D data.

## 4D-to-video rendering

In Medusa, we try to make it as easy as possible to render 4D reconstruction data as a video. As you might have seen in the [quickstart](../getting_started/quickstart), you can use a `VideoRenderer` object for this. Note that this renderer is only available if you have [pytorch3d](https://pytorch3d.org/) installed, which is unfortunately not possible on Windows at the moment.

In [ ]:
from medusa.render import VideoRenderer

The class constructor takes three arguments &mdash; `shading`, `lights`, and `loglevel`. We'll ignore the `lights` argument, which we'll discuss later.

The `shading` argument can be either "flat", which creates a faceted look, or "smooth", which creates a smoother surface using [Phong shading](https://en.wikipedia.org/wiki/Phong_shading). We'll use smooth shading for now and set the loglevel to 'WARNING' (which does not output a progress bar which clutters the website):

In [ ]:
renderer = VideoRenderer(shading='smooth', loglevel='WARNING')

The renderer expects the 4D reconstruction data to be wrapped in a `Data4D` object (see [data representation tutorial](./data_representation)). Let's load in the 4D reconstruction (by the 'emoca-coarse' model) from our default video:

In [ ]:
from medusa.data import get_example_h5
data_4d = get_example_h5(load=True, model='emoca-coarse')
# We'll slice the Data4D object this way (resulting in only 50 frames) so that the 
# rendering is a bit faster for this tutorial
data_4d = data_4d[:50]

To render the 4D data to a video, you use the `VideoRenderer` object's `__call__` method (i.e., calling `renderer()`). This method has two mandatory arguments:

* `f_out`: path where the video will be saved
* `data`: the `Data4D` object

Additionally, this method accepts two optional arguments:

* `video`: path to original video
* `overlay`: colors to project onto the vertices before rendering

If you set the `video` argument to the original video associated with the 4D reconstruction, then the video will be used as a background on which the 4D data is rendered. The default value is `None` (in case the data will be rendered on a white background). 

Let's render our data on top of the video!

In [ ]:
from medusa.data import get_example_video
from IPython.display import Video

vid = get_example_video()
f_out = './smooth.mp4'

renderer(f_out, data_4d, video=vid)

# Show result
Video(f_out, embed=True)

One way to make the visualization a little nicer is by only rendering the face (rather than the full head). To do so, you can use the `apply_vertex_mask` method from the `Data4D` object with the `name` argument set to `'face'`:

In [ ]:
data_4d.apply_vertex_mask('face')

This method basically removes all non-face vertices from the mesh, leaving us with 1787 vertices (instead of the original 5023):

In [ ]:
tuple(data_4d.v.shape)

Now, let's re-render the data (which is a lot faster now too, as it has to work with fewer vertices):

In [ ]:
renderer(f_out, data_4d, video=vid)
Video(f_out, embed=True)

### Overlays

So far, we only rendered the face as a grayish, untextured shape. We can, however, give it a different uniform color or specific color per vertex. To do so, we can give the `__call__` function an additional argument, `overlay`, a tensor with a single color per vertex. Colors need to be represented as RGB float values ranging from 0 to 1, so overlay should be a $V \times 3$ tensor. 

To demonstrate, let's create an overlay that'll make the face bright red, which corresponds to an RGB value of [1., 0., 0.]:

In [ ]:
import torch

V = data_4d.v.shape[1]
vertex_colors = torch.zeros((V, 3), device=data_4d.device)
vertex_colors[:, 0] = 1.  # set 'red' channel to 1

tuple(vertex_colors.shape)

Let's render the video again, but now with `vertex_colors` used for the `overlay` argument:

In [ ]:
f_out = './test.mp4'
renderer(f_out, data_4d, overlay=vertex_colors)
Video(f_out, embed=True)

Note that we don't have to use the same colors for each time point! We can also create an overlay of shape $N$ (nr of frames) $\times V \times 3$, which specifies a specific color for each frame (and each vertex). To demonstrate, we'll generate random RGB values for each frame and vertex, creating quite a trippy visualization!

In [ ]:
N = data_4d.v.shape[0]
vertex_colors = torch.rand(N, V, 3, device='cuda')
renderer(f_out, data_4d, overlay=vertex_colors)
Video(f_out, embed=True)

Now suppose that we would like to color the face along a more interesting feature, like the amount of local movement relative to the first frame. We'll walk you through this example step by step.

First, frame-by-frame movement can be computed as follows:

In [ ]:
# To only show local deviations, we can use the to_local() method which projects out any "global" movement
data_4d.to_local()
local_mov = data_4d.v - data_4d.v[0, :, :]
tuple(local_mov.shape)

In [ ]:
from medusa.geometry import compute_vertex_normals
normals = compute_vertex_normals(data_4d.v[0], data_4d.tris)
local_mov_proj = (normals * local_mov).sum(dim=2)
from matplotlib.colors import CenteredNorm
from matplotlib import colormaps

norm = CenteredNorm(vcenter=0.)
cmap = colormaps['bwr']
local_mov_proj = torch.as_tensor(cmap(norm(local_mov_proj.cpu().numpy()))[:, :, :3], device=data_4d.v.device, dtype=torch.float32)
data_4d.to_world()
renderer(f_out, data_4d, overlay=local_mov_proj)
Video(f_out, embed=True)

## 3D-to-image rendering

ToDo: explanation of the low(er)-level `PytorchRenderer` class that performs 3D-to-image rendering